<a href="https://colab.research.google.com/github/lvvillarroel/relation-discover/blob/master/preprocessing_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/drive')

Mounted at /drive


In [2]:
import json

with open('majors.json', 'r', encoding='utf-8') as json_majors:
    j = json_majors.read()
    majors = json.loads(j)

MAJOR = 'Investigación Operativa'
MAJOR_COURSES = majors[MAJOR]
# ['IIC2233', 'IIC2413', 'IIC2343', 'IIC1253', 'IIC2713', 'IIC2143', 'IIC2333', 'IIC2133', 'IIC2613', 'IIC2154']

In [4]:
df = pd.read_csv('{}.csv'.format(MAJOR))
df = df.drop(columns=['Unnamed: 0'])

In [6]:
# Agrupar los alumnos, luego por cada uno comenzar desde 0
def create_semcolumns(df, filter_rows=[], filter=False):
  df_students = df.groupby('ID')
  major_sem = []
  orden = []
  for studend, group in df_students:
    mins = group['DURACIÓN'].min()
    g = group.sort_values('DURACIÓN')
    order = 0
    last_value = 0
    for row in group.itertuples():
      d = row.DURACIÓN
      c = row.SIGLA
      msem = d - mins + 1
      if (not filter and d > last_value) or (filter and not (c in filter_rows) and d > last_value):
        order+=1
      last_value = d
      orden.append(order)
      major_sem.append(msem)
  if filter:
    df.insert(3, 'MOFILTRO', orden, True)
  else:
    df.insert(3, 'MSEM', major_sem, True)
    df.insert(4, 'MORDEN', orden, True)
  return df

In [7]:
# Permite crear los dataframes en formato de "vector", retorna el vector de "SEMESTRES" y de "ORDENES"
def create_vectordf(df, mc=MAJOR_COURSES, filter_rows=[], filter=False):
  matrix_order = []
  matrix_sem = []
  df_students = df.groupby('ID')
  for student, group in df_students:
    sem = [0 for x in range(len(mc))]
    ord = [0 for x in range(len(mc))]
    for row in group.itertuples():
      c = row.SIGLA
      if filter and (c in filter_rows):
        continue
      ix = mc.index(c)
      if filter:
        ord[ix] = row.MOFILTRO
      else:
        sem[ix] = row.MSEM
        ord[ix] = row.MORDEN
    matrix_sem.append(sem)
    matrix_order.append(ord)
  df_orden = pd.DataFrame(data=matrix_order, columns=mc)
  df_sem = pd.DataFrame(data=matrix_sem, columns=mc)
  return df_sem, df_orden

In [8]:
# Permite ver los cursos problematicos (std sobre cierto limite)
def rows2filter(df, limit):
  dev = df.std()
  print(dev)
  d = dev.where(dev > limit).dropna()
  return d.index.to_list()

In [9]:
# Obtener los resultados
new_df = create_semcolumns(df)
new_df
# df_sem, df_orden = create_vectordf(new_df)
# f = rows2filter(df_orden, 0.9)
# df_sem_filtered = df_sem.drop(columns=f)
# df_ord_filtered1 = df_orden.drop(columns=f)
# df_fil = create_semcolumns(df, filter_rows=f, filter=True)
# MC = [x for x in MAJOR_COURSES if x not in f]
# _ , df_ord_filtered2 = create_vectordf(df_fil, mc=MC, filter_rows=f, filter=True)

,ID,SIGLA,NOTA FINAL,MSEM,MORDEN,AÑO,DURACIÓN,SEMESTRE,PPA GLOBAL
0,5,ICS1113,4.4,1,1,2015,3,1,4.82
1,5,ICS2523,4.7,2,2,2015,4,2,4.82
2,5,ICS2123,4.1,4,3,2016,6,2,4.82
3,5,ICS2562,5.7,4,3,2016,6,2,4.82
4,5,ICS2121,4.8,5,4,2017,7,1,4.82
...,...,...,...,...,...,...,...,...,...
2755,4677,ICS2813,4.8,4,4,2017,7,1,4.90
2756,4677,ICS2121,4.9,4,4,2017,7,1,4.90
2757,4677,ICS2562,5.7,4,4,2017,7,1,4.90
2758,4677,ICS2613,4.9,5,5,2017,8,2,4.90


In [19]:
# df_sem, df_orden = create_vectordf(new_df)
# df_orden
# f = rows2filter(df_orden, 0.9)
# df_sem_filtered = df_sem.drop(columns=f)
# df_ord_filtered1 = df_orden.drop(columns=f)
# df_fil = create_semcolumns(df, filter_rows=f, filter=True)
MC = [x for x in MAJOR_COURSES if x not in f]
_ , df_ord_filtered2 = create_vectordf(df_fil, mc=MC, filter_rows=f, filter=True)
df_ord_filtered2

,ICS1113,ICS2523,ICS2123,ICS2613,ICS2122,ICS2562
0,1,2,3,4,5,3
1,1,2,3,5,4,3
2,1,2,3,4,5,3
3,1,2,3,3,4,2
4,1,2,2,2,2,2
...,...,...,...,...,...,...
340,1,2,3,3,4,2
341,2,3,3,3,3,1
342,1,2,3,4,4,2
343,1,2,2,3,3,1


In [22]:
# Pasar los datos a csv
# df_sem.to_csv('/drive/My Drive/Colab Notebooks/data/{} - sem v.csv'.format(MAJOR), index=False, index_label=False)
# df_orden.to_csv('/drive/My Drive/Colab Notebooks/data/{} - orden v.csv'.format(MAJOR), index=False, index_label=False)
df_sem_filtered.to_csv('/drive/My Drive/Colab Notebooks/data/{} - sem fil v.csv'.format(MAJOR), index=False, index_label=False)
# df_ord_filtered1.to_csv('/drive/My Drive/Colab Notebooks/data/{} - orden fil1 v.csv'.format(MAJOR), index=False, index_label=False)
df_ord_filtered2.to_csv('/drive/My Drive/Colab Notebooks/data/{} - orden fil2 v.csv'.format(MAJOR), index=False, index_label=False)

In [17]:
df_fil.head(30)

,ID,SIGLA,NOTA FINAL,MOFILTRO,MSEM,MORDEN,AÑO,DURACIÓN,SEMESTRE,PPA GLOBAL
0,5,ICS1113,4.4,1,1,1,2015,3,1,4.82
1,5,ICS2523,4.7,2,2,2,2015,4,2,4.82
2,5,ICS2123,4.1,3,4,3,2016,6,2,4.82
3,5,ICS2562,5.7,3,4,3,2016,6,2,4.82
4,5,ICS2121,4.8,3,5,4,2017,7,1,4.82
5,5,ICS2613,4.4,4,7,5,2018,9,1,4.82
6,5,ICS2813,5.2,4,7,5,2018,9,1,4.82
7,5,ICS2122,5.9,5,8,6,2018,10,2,4.82
8,14,ICS1113,4.8,1,1,1,2015,4,2,5.28
9,14,ICS2523,5.7,2,2,2,2016,5,1,5.28


In [ ]:
# Filtrar + primer y último curso
FIL = ['IIC2233']
F = ['IIC2713', 'IIC2133', 'IIC2613', 'IIC2413', 'IIC2343', 'IIC1253', 'IIC2143', 'IIC2333', 'IIC2154']
df_fil2 = create_semcolumns(df, filter_rows=F, filter=True)
_ , df_ord_fil2 = create_vectordf(df_fil2, mc=FIL, filter_rows=F, filter=True)
df_ord_fil2

,IIC2233
0,1
1,1
2,1
3,1
4,1
...,...
87,1
88,1
89,1
90,1


In [ ]:
# df_ord_fil2.head(30)
df_orden.head(30)

,IIC2233,IIC2413,IIC2343,IIC1253,IIC2713,IIC2143,IIC2333,IIC2133,IIC2613,IIC2154
0,1,2,3,4,4,5,6,7,7,8
1,1,2,2,3,3,4,3,4,6,5
2,2,3,4,3,1,5,5,5,4,6
3,1,2,3,3,3,5,4,4,4,6
4,1,2,2,2,1,2,3,4,4,5
5,1,3,2,2,5,4,3,5,4,5
6,1,2,3,2,1,3,4,3,5,5
7,1,2,1,2,4,3,2,3,6,5
8,1,3,2,4,1,6,6,6,5,7
9,1,2,2,2,2,3,4,3,3,4
